## Fazendo import das dependências

In [1]:
from BHA_functions import asyncSimulations_Linux
from BHA_functions import DataCollector
from BHA_functions import GraphicGenerator
from datetime import datetime

## Definindo as constantes das simulações

In [2]:
PATH = 'Simulations_Data/default_simulations'
CORES = 12
RUNS_PER_POINT = 1

TARGET = (True, False)
GRAPHIC_POINTS = 7

NETWORK_PROB = 0.8
TOPOLOGY = 'Grade'
INTENSITY_VARIATION = 0.1
BHA_NODES = (1, 3, 5)

## Rodando a simulação sem Black Hole

In [ ]:
intensity = []
for c in range(1, GRAPHIC_POINTS+1):
    intensity.append(NETWORK_PROB - c*INTENSITY_VARIATION)

# Default params
simulations_params = {
            'runs':RUNS_PER_POINT,
            'topology':TOPOLOGY,
            'number_nodes':20,
            'rows':4,
            'columns':3,
            'prob_edge_creation':0.1,
            'edges_to_attach':3,
            'entanglements_replanished':10,
            'requests':100,
            'attempts_per_request':2,
            'network_prob':NETWORK_PROB,
            'num_black_holes':0,
            'black_hole_prob':intensity[0],
            'black_hole_target':TARGET
        }

# Runing no BHA simulation
default_simulation = asyncSimulations_Linux(cores=CORES, **simulations_params)
default_simulation.save(file_name=f'{PATH}/Default_Network/default_network')

## Realizando todas as simulações com Black Holes

In [ ]:
for target in TARGET:
    for bha_nodes in BHA_NODES:
        for i in range(0, GRAPHIC_POINTS):
            start = datetime.now()

            # Updating the params
            simulations_params['num_black_holes'] = bha_nodes
            simulations_params['black_hole_prob'] = intensity[i]
            simulations_params['black_hole_target'] = target

            # Running simulation
            simulations_dc = asyncSimulations_Linux(cores=CORES, **simulations_params)

            # Saving CSV file
            simulations_dc.save(file_name=f'{PATH}/Target-{target}/{bha_nodes}nodes/point{i}')

            # BenchMark
            print(f"As {simulations_params['runs']} simulações com target {target} e com {bha_nodes} atacantes finalizaram no tempo de: {datetime.now()-start}")
            print('-='*50)

## Coletando os dados diretamente dos CSVs de cada simulação

In [5]:
default_network = DataCollector()
default_network.get_DataFrame_csv(f'{PATH}/Default_Network/default_network.csv')

datacollectors_list_target_on = []
datacollectors_list_target_off = []
for target in TARGET:
    for bha_nodes in BHA_NODES:
        temp_datacollectors = []
        for i in range(0, GRAPHIC_POINTS):
            temp_dc = DataCollector()
            temp_dc.get_DataFrame_csv(f'{PATH}/Target-{target}/{bha_nodes}nodes/point{i}.csv')
            temp_datacollectors.append(temp_dc)
        if target:
            datacollectors_list_target_on.append(tuple(temp_datacollectors))
        else:
            datacollectors_list_target_off.append(tuple(temp_datacollectors))

## Criando uma função genérica para plotar os gráficos

In [6]:
def plotGraphic(
        dataCollectors_list: list[tuple[DataCollector]],
        default_diff: DataCollector, 
        intensity_variation: float,
        bha_nodes: tuple,
        y_column_name: str, 
        title: str, 
        x_label: str, 
        y_label: str) -> None:
    """
    Will show the simulations graph

    Args:
        dataCollectors_list (required): List with tuples of DataCollectors
        default_diff (required): Pass the network without black holes to compare, if None don't affect the result
        intensity_variation (required): Variation of intensity per point
        bha_nodes (required): Tuple with number of nodes
        y_column_name (required): Name of column data
        title (required): Title of graph
        x_label (required): X axis label
        y_label (required): Y axis label
    """
    
    graphicGen = GraphicGenerator()

    for pos, datacollectors in enumerate(dataCollectors_list):
        graphicGen.add_on_plot(
            plot_label=f'{bha_nodes[pos]} nós', 
            x_column=(0.1, intensity_variation), 
            y_column_name=y_column_name, 
            y_standard_deviation=False,
            dc=datacollectors,
            default_diff=default_diff)

    graphicGen.show_plot(
        title=title,
        x_label=x_label, 
        y_label=y_label)

## Gerando o gráfico da Taxa de Sucesso sem Target

In [ ]:
plotGraphic(
    dataCollectors_list=datacollectors_list_target_off,
    default_diff=default_network,
    intensity_variation=INTENSITY_VARIATION,
    bha_nodes=BHA_NODES,
    y_column_name="Success Tax",
    title="Diferença na Taxa de Sucesso vs Intensidade do Ataque (Target off)",
    x_label='Intensidade',
    y_label='Diferença na Taxa de Sucesso'
    )

## Gerando o gráfico da Taxa de Sucesso com Target

In [ ]:
plotGraphic(
    dataCollectors_list=datacollectors_list_target_on,
    default_diff=default_network,
    intensity_variation=INTENSITY_VARIATION,
    bha_nodes=BHA_NODES,
    y_column_name="Success Tax",
    title="Diferença na Taxa de Sucesso vs Intensidade do Ataque (Target on)",
    x_label='Intensidade',
    y_label='Diferença na Taxa de Sucesso'
    )

## Gerando o Gráfico da Taxa de Eprs Usados sem Target

In [ ]:
plotGraphic(
    dataCollectors_list=datacollectors_list_target_off,
    default_diff=default_network,
    intensity_variation=INTENSITY_VARIATION,
    bha_nodes=BHA_NODES,
    y_column_name="Used Eprs",
    title="Diferença na Taxa de Eprs Usados vs Intensidade do Ataque (Target off)",
    x_label='Intensidade',
    y_label='Diferença na Taxa de Eprs Usados'
)

## Gerando o Gráfico da Taxa de Eprs Usaddos com Target

In [ ]:
plotGraphic(
    dataCollectors_list=datacollectors_list_target_on,
    default_diff=default_network,
    intensity_variation=INTENSITY_VARIATION,
    bha_nodes=BHA_NODES,
    y_column_name="Used Eprs",
    title="Diferença na Taxa de Eprs Usados vs Intensidade do Ataque (Target on)",
    x_label='Intensidade',
    y_label='Diferença na Taxa de Eprs Usados'
)